# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree.
* Test your function on the tuple `('yes',31,'good')`,

In [23]:
def exercise1(t):
    if(t[0] == 'yes'):
        if(t[1] <= 29.5):
            result = (t,'less')
        elif(t[1] > 29.5):
            result = (t,'more')
    else:
        if(t[2] == 'good'):
            result = (t, 'less')
        elif(t[2] == 'poor'):
            result = (t, 'more')
    return result
exercise1(('yes', 31, 'good'))

(('yes', 31, 'good'), 'more')

## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples.

In [24]:
def exercise2(filename):
    result = []
    f = open(filename, 'r')
    for line in f:
        string_list = str.split(line[:-1], ',')
        string_list[1] = int(string_list[1])
        result = result + [tuple(string_list)]
    return result
exercise2('health-test.txt')

[('yes', 21, 'poor'),
 ('no', 50, 'good'),
 ('no', 23, 'good'),
 ('yes', 45, 'poor'),
 ('yes', 51, 'good'),
 ('no', 60, 'good'),
 ('no', 15, 'poor'),
 ('no', 18, 'good')]

## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and compute the percentage of them that are classified as "more risk".

In [33]:
def exercise3():
    test_set = exercise2('health-test.txt')
    return len(filter(lambda x: exercise1(x)[1] == 'more', test_set))/float(len(test_set))
exercise3()

0.375

## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [43]:
def exercise4(filename):
    f = open(filename, 'r')
    result = []
    for line in f:
        string_list = str.split(line[:-1], ',')
        string_list[1] = int(string_list[1])
        attribute_list = string_list[0:3]
        
        result = result + [(tuple(attribute_list), string_list[3])]
    return result
exercise4('health-train.txt')

[(('yes', 54, 'good'), 'less'),
 (('no', 55, 'good'), 'less'),
 (('no', 26, 'good'), 'less'),
 (('yes', 40, 'good'), 'more'),
 (('yes', 25, 'poor'), 'less'),
 (('no', 13, 'poor'), 'more'),
 (('no', 15, 'good'), 'less'),
 (('no', 50, 'poor'), 'more'),
 (('yes', 33, 'good'), 'more'),
 (('no', 35, 'good'), 'less'),
 (('no', 41, 'good'), 'less'),
 (('yes', 30, 'poor'), 'more'),
 (('no', 39, 'poor'), 'more'),
 (('no', 20, 'good'), 'less'),
 (('yes', 18, 'poor'), 'less'),
 (('yes', 55, 'good'), 'more')]

## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a,b) = (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes',31,'good')`

In [56]:
def exercise5a(x):
    train_set = exercise4('health-train.txt')
    ret = [x,100]
    for i in train_set:
        d_tmp = (x[0] != i[0][0]) + ((x[1] - i[0][1])/50.0)**2 + (x[2] != i[0][2])
        if d_tmp < ret[1]:
            ret = [i, d_tmp]
    return (x, ret[0][1])
exercise5a(('yes', 31, 'good'))

(('yes', 31, 'good'), 'more')

* Apply both the decision tree and nearest neighbor classifiers on the test set, and find the data point(s) for which the two classifiers disagree, and the fraction of the time it happens.

In [58]:
def exercise5b():
    test_set = exercise2('health-test.txt')
    disagreement_list = filter(lambda x: exercise1(x) != exercise5a(x), test_set)
    disagreement_ratio = len(disagreement_list)/float(len(test_set))
    return (disagreement_list, disagreement_ratio)
exercise5b()

([('yes', 51, 'good')], 0.125)

One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0]-b[0])**2+((a[1]-b[1])/50.0)**2+(a[2]-b[2])**2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [94]:
class NearestMeanClassifier:
    
    # Training method that takes as input a dataset
    # and produces two internal vectors corresponding
    # to the mean of each class.
    def train(self,dataset):
        less_counts = len(filter(lambda x: x[1] == 'less', dataset))
        more_counts = len(filter(lambda x: x[1] == 'more', dataset))
        less_attributes = [0,0,0]
        more_attributes = [0,0,0]
        for x in dataset:
            if(x[1] == 'less'):
                if(x[0][0] == 'yes'):
                    less_attributes[0] = less_attributes[0] + 1
                less_attributes[1] = less_attributes[1] + x[0][1]
                if(x[0][2] == 'poor'):
                    less_attributes[2] = less_attributes[2] + 1
            if(x[1] == 'more'):
                if(x[0][0] == 'yes'):
                    more_attributes[0] = more_attributes[0] + 1
                more_attributes[1] = more_attributes[1] + x[0][1]
                if(x[0][2] == 'poor'):
                    more_attributes[2] = more_attributes[2] + 1
        less_attributes_mean = (less_attributes[0]/float(less_counts), 
                                less_attributes[1]/float(less_counts), 
                                less_attributes[2]/float(less_counts))
        more_attributes_mean = (more_attributes[0]/float(more_counts), 
                                more_attributes[1]/float(more_counts), 
                                more_attributes[2]/float(more_counts))
        self.mean =  {'less': less_attributes_mean, 'more': more_attributes_mean}
                
    
    # Prediction method that takes as input a new data
    # point and predicts it to belong to the class with
    # nearest mean.
    def predict(self,x):
        x_list = list(x)
        if(x_list[0] == 'yes'):
            x_list[0] = 1.0
        else:
            x_list[0] = 0.0
        if(x_list[2] == 'good'):
            x_list[2] = 0.0
        else:
            x_list[2] = 1.0
        l_m = self.mean['less']
        m_m = self.mean['more']
        d_to_less_mean = (x_list[0] - l_m[0])**2 + ((x_list[1] - l_m[1])/50.0)**2 + (x_list[2] - l_m[2])**2
        d_to_more_mean = (x_list[0] - m_m[0])**2 + ((x_list[1] - m_m[1])/50.0)**2 + (x_list[2] - m_m[2])**2
        if(d_to_less_mean < d_to_more_mean):
            return (x, 'less')
        else:
            return (x, 'more')

* Build an object of class `NearestMeanClassifier`, train it on the training data, and print the mean vector for each class.

In [95]:
nmc = NearestMeanClassifier()
train_set = exercise4('health-train.txt')
nmc.train(train_set)
print(nmc.mean)

{'more': (0.5714285714285714, 37.142857142857146, 0.5714285714285714), 'less': (0.3333333333333333, 32.111111111111114, 0.2222222222222222)}


* Predict the test data using the nearest mean classifier and print all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree.

In [109]:
def exercise6b():
    test_set = exercise2('health-test.txt')
    result = []
    for i in test_set:
        if(nmc.predict(i) == exercise5a(i) and exercise5a(i) == exercise1(i)):
            result = result + [nmc.predict(i)]
    return result
exercise6b()


[(('no', 50, 'good'), 'less'),
 (('no', 23, 'good'), 'less'),
 (('yes', 45, 'poor'), 'more'),
 (('no', 60, 'good'), 'less'),
 (('no', 15, 'poor'), 'more'),
 (('no', 18, 'good'), 'less')]